# Store Sales - Multiple Features Forecasting

https://www.kaggle.com/competitions/store-sales-time-series-forecasting

***In this project, forecasting will be processed with considering 'promotion' featureas well.<br>
Also, we will implement forecasting per each product family***

The evaluation metric for this competition is Root Mean Squared Logarithmic Error.

The RMSLE is calculated as:
$\sqrt{ \frac{1}{n} \sum_{i=1}^n \left(\log (1 + \hat{y}_i) - \log (1 + y_i)\right)^2}$
where:

𝑛 is the total number of instances,<br>
𝑦̂ 𝑖 is the predicted value of the target for instance (i),<br>
𝑦𝑖 is the actual value of the target for instance (i), and,<br>
log is the natural logarithm.

The training data; <br>
***store_nbr*** identifies the store at which the products are sold.<br>
***family*** identifies the type of product sold.<br>
***sales*** gives the total sales for a product family at a particular store at a given date.
Fractional values are possible since products can be sold in fractional units (1.5 kg of cheese, for instance, as opposed to 1 bag of chips).<br>
***onpromotion*** gives the total number of items in a product family that were being promoted at a store at a given date.

## Blue print

1. Investigate the dataset. (unique values, data type etc)
2. How to numerize *family* features?
3. How to convert *date* to time features?
4. Split *train* dataset to *ourtrain* and *ourtest* for pre-validation.
5. Apply various ML models. (Trend, Periodtogram, Cycles, Hybrid)
6. Choose the best model and apply to our test set.
7. Apply and make csv file for submition.


## Preprocessing

In [12]:
# Import packages
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
import datetime
import math
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# Ignore Future Warning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [13]:
# Load dataset
train = pd.read_csv('train.csv', parse_dates=["date"])
test = pd.read_csv('test.csv', parse_dates=["date"])

In [14]:
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


#### - Error function : 
$\sqrt{ \frac{1}{n} \sum_{i=1}^n \left(\log (1 + \hat{y}_i) - \log (1 + y_i)\right)^2}$

𝑛 is the total number of instances,<br>
𝑦̂ 𝑖 is the predicted value of the target for instance (i),<br>
𝑦𝑖 is the actual value of the target for instance (i)

In [15]:
# Error Function (RMSLE)
def error(y_p, y_t):
    pred_log = np.array([math.log(i+1) for i in np.nditer(y_p)])
    act_log = np.array([math.log(i+1) for i in np.nditer(y_t)])
    dum_error = sum((pred_log - act_log)**2)/len(pred_log)
    linear_error = np.power(dum_error, 1/2)
    return round(linear_error, 4)

## 1. Data investigation

#### - train dataset
* shape : 3000888 × 6
* null : none
<br><br>
* *date* : timestamp. 2013-01-01 ~ 2017-08-15
* *store_nbr* : numpy. 1 ~ 54
* *family* : str. ['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS',
       'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS',
       'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE',
       'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES',
       'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE',
       'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE',
       'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY',
       'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES',
       'SEAFOOD']
* *sales* : numpy. 0 ~ 124717
* *onpromotion* : numpy. 0 ~ 741


#### - Correlation

In [16]:
# Check correlation
train.corr()

,id,store_nbr,sales,onpromotion
id,1.000000,0.000301,0.085784,0.206260
store_nbr,0.000301,1.000000,0.041196,0.007286
sales,0.085784,0.041196,1.000000,0.427923
onpromotion,0.206260,0.007286,0.427923,1.000000


## 2. One Hot Encode *'family'* features

In [17]:
# Integer encode
label_encoderF = LabelEncoder()
integer_encodedF = label_encoderF.fit_transform(train['family'])

# Binary encode
onehot_encoderF = OneHotEncoder(sparse=False)
integer_encodedF = integer_encodedF.reshape(len(integer_encodedF), 1)
onehot_encodedF = onehot_encoderF.fit_transform(integer_encodedF)
# print(onehot_encodedF.shape, '\n', onehot_encodedF)

In [18]:
# Rename
onehot_encodedF = pd.DataFrame(onehot_encodedF)
onehot_encodedF = onehot_encodedF.rename(columns = {0:'AUTOMOTIVE', 1:'BABY CARE', 2:'BEAUTY', 3:'BEVERAGES', 
        4:'BOOKS', 5:'BREAD/BAKERY', 6:'CELEBRATION', 7:'CLEANING', 8:'DAIRY', 9:'DELI', 10:'EGGS',
        11:'FROZEN FOODS', 12:'GROCERY I', 13:'GROCERY II', 14:'HARDWARE',
        15:'HOME AND KITCHEN I', 16:'HOME AND KITCHEN II', 17:'HOME APPLIANCES',
        18:'HOME CARE', 19:'LADIESWEAR', 20:'LAWN AND GARDEN', 21:'LINGERIE',
        22:'LIQUOR,WINE,BEER', 23:'MAGAZINES', 24:'MEATS', 25:'PERSONAL CARE',
        26:'PET SUPPLIES', 27:'PLAYERS AND ELECTRONICS', 28:'POULTRY',
        29:'PREPARED FOODS', 30:'PRODUCE', 31:'SCHOOL AND OFFICE SUPPLIES',
        32:'SEAFOOD'})

In [19]:
# Add to train dataset
train = pd.concat([train, onehot_encodedF], axis=1) # Combine
train = train.drop(columns=['family'])              # Drop string values
print(train.shape)
train.head()

(3000888, 38)


,id,date,store_nbr,sales,onpromotion,AUTOMOTIVE,BABY CARE,BEAUTY,BEVERAGES,BOOKS,...,MAGAZINES,MEATS,PERSONAL CARE,PET SUPPLIES,PLAYERS AND ELECTRONICS,POULTRY,PREPARED FOODS,PRODUCE,SCHOOL AND OFFICE SUPPLIES,SEAFOOD
0,0,2013-01-01,1,0.0,0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2013-01-01,1,0.0,0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2013-01-01,1,0.0,0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2013-01-01,1,0.0,0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2013-01-01,1,0.0,0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 3. One Hot Encode *'store_nbr'* features

In [20]:
# Convert to numpy
integer_encodedS = np.array(train['store_nbr'])

# Binary encode
onehot_encoderS = OneHotEncoder(sparse=False)
integer_encodedS = integer_encodedS.reshape(len(integer_encodedS), 1)
onehot_encodedS = onehot_encoderS.fit_transform(integer_encodedS)
# print(onehot_encodedS.shape, '\n', onehot_encodedS)

In [21]:
# Add to train dataset
onehot_encodedS = pd.DataFrame(onehot_encodedS)
train = pd.concat([train, onehot_encodedS], axis=1) # Combine
train = train.drop(columns=['store_nbr'])           # Drop string values
print(train.shape)
train.head()

(3000888, 91)


,id,date,sales,onpromotion,AUTOMOTIVE,BABY CARE,BEAUTY,BEVERAGES,BOOKS,BREAD/BAKERY,...,44,45,46,47,48,49,50,51,52,53
0,0,2013-01-01,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2013-01-01,0.0,0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2013-01-01,0.0,0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2013-01-01,0.0,0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2013-01-01,0.0,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 4. Convert *'date'* to time features

In [ ]:
# Make 'date' as an index
train = pd.DataFrame(train)
train = train.set_index('date')
train = train.to_period('D')
print(train.shape, train.head())

In [25]:
# Split 'date' into detailed features
train["day"] = train.index.dayofweek
train["week"] = train.index.week
train["dayofyear"] = train.index.dayofyear
train["year"] = train.index.year

print(train.shape)
train.head()

(3000888, 94)


,id,sales,onpromotion,AUTOMOTIVE,BABY CARE,BEAUTY,BEVERAGES,BOOKS,BREAD/BAKERY,CELEBRATION,...,48,49,50,51,52,53,day,week,dayofyear,year
date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,2013
2013-01-01,1,0.0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,2013
2013-01-01,2,0.0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,2013
2013-01-01,3,0.0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,2013
2013-01-01,4,0.0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,2013


## 5. Split *train* dataset into *ourtrain* and *ourtest*

In [33]:
ourtrain = train[train.index < '2017-01-01']    # 2013-01-01 ~ 2016-12-31
ourtest = train[train.index > '2016-12-31']     # 2017-01-01 ~ 2017-08-15

print(len(ourtrain)+len(ourtest))

3000888


In [35]:
ourtrain.head()

,id,sales,onpromotion,AUTOMOTIVE,BABY CARE,BEAUTY,BEVERAGES,BOOKS,BREAD/BAKERY,CELEBRATION,...,48,49,50,51,52,53,day,week,dayofyear,year
date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,2013
2013-01-01,1,0.0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,2013
2013-01-01,2,0.0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,2013
2013-01-01,3,0.0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,2013
2013-01-01,4,0.0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,2013


In [36]:
ourtest.head()

,id,sales,onpromotion,AUTOMOTIVE,BABY CARE,BEAUTY,BEVERAGES,BOOKS,BREAD/BAKERY,CELEBRATION,...,48,49,50,51,52,53,day,week,dayofyear,year
date,,,,,,,,,,,,,,,,,,,,,
2017-01-01,2596374,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6,52,1,2017
2017-01-01,2596375,0.0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6,52,1,2017
2017-01-01,2596376,0.0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6,52,1,2017
2017-01-01,2596377,0.0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6,52,1,2017
2017-01-01,2596378,0.0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6,52,1,2017


## 6. Apply various ML models

## 7. Proceed forecasting with the best model

## 8. Generate csv file